# <center> <img src="../../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> Computer Systems Engineering  </center>
---
### <center> Big Data Processing </center>
---
#### <center> **Autumn 2025** </center>
---

**Lab 05**: Data pipeline with Neo4j

**Date**: October 2nd 2025

**Student Name**: Francisco Delgado

**Professor**: Pablo Camarillo Ramirez

# Dataset description

# Data ingestion

In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Examples on SparkSQL") \
    .master("spark://9eae4bedccb3:7077") \
    .config("spark.jars.packages", "org.neo4j:neo4j-connector-apache-spark_2.13:5.3.10_for_spark_3") \
    .config("spark.ui.port", "4040") \
    .getOrCreate()

sc = spark.sparkContext
sc.setLogLevel("ERROR")

:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /root/.ivy2.5.2/cache
The jars for the packages stored in: /root/.ivy2.5.2/jars
org.neo4j#neo4j-connector-apache-spark_2.13 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-d2561f40-b4de-46e9-a680-f39ffe050a9c;1.0
	confs: [default]
	found org.neo4j#neo4j-connector-apache-spark_2.13;5.3.10_for_spark_3 in central
	found org.neo4j#neo4j-connector-apache-spark_2.13_common;5.3.10_for_spark_3 in central
	found org.neo4j#caniuse-core;1.3.0 in central
	found org.neo4j#caniuse-api;1.3.0 in central
	found org.jetbrains.kotlin#kotlin-stdlib;2.1.20 in central
	found org.jetbrains#annotations;13.0 in central
	found org.neo4j#caniuse-neo4j-detection;1.3.0 in central
	found org.neo4j.driver#neo4j-java-driver-slim;4.4.21 in central
	found org.reactivestreams#reactive-streams;1.0.4 in central
	found io.netty#netty-handler;4.1.

In [2]:
from FrankModule.spark_utils import SparkUtils

# === Definir esquema de las columnas ===
schema_columns = [
    ("id", "string"),
    ("name", "string"),
    ("genre", "string"),
    ("artists", "string"),
    ("album", "string"),
    ("popularity", "int"),
    ("duration_ms", "long"),
    ("explicit", "string"),
]

songs_schema = SparkUtils.generate_schema(schema_columns)

# === Ruta corregida ===
csv_path = "spark/notebooks/data/spotify_tracks/spotify_tracks.csv"

# === Lectura del CSV con el esquema ===
df_songs = spark.read \
    .option("header", "true")\
    .schema(songs_schema)
    .csv(csv_path)


# === Mostrar muestra ===
df_songs.show(5, truncate=False)
df_songs.printSchema()



IndentationError: unexpected indent (1751331892.py, line 22)

In [ ]:
from pyspark.sql.functions import col

csv_path = "data/spotify_tracks/spotify_tracks.csv"

df = (spark.read
      .option("header", "true")
      .option("inferSchema", "true")
      .csv(csv_path))

df.printSchema()
df.show(5, truncate=False)
df.count()


# Transformations

In [ ]:
from pyspark.sql.functions import split, explode, trim

df_norm = (df
    .withColumn("artists_list", split(col("artists"), ","))
)

song_artist = (df_norm
    .withColumn("artist_name", trim(explode(col("artists_list"))))
    .select(
        col("id").alias("song_id"),
        col("name").alias("song_name"),
        col("popularity"),
        col("duration_ms"),
        col("explicit"),
        col("genre"),
        col("album"),
        col("artist_name")
    )
    .dropna(subset=["song_id", "artist_name"])
)

song_artist.show(5, truncate=False)
song_artist.count()


In [ ]:
# Nodos de canciones
songs_df = (song_artist
    .select(
        col("song_id").alias("id"),
        col("song_name").alias("name"),
        "popularity", "duration_ms", "explicit"
    )
    .dropDuplicates(["id"])
)

# Nodos de artistas
artists_df = (song_artist
    .select(col("artist_name").alias("name"))
    .dropna()
    .dropDuplicates(["name"])
)

# (Opcional) Nodos de género
genres_df = (song_artist
    .select(trim(col("genre")).alias("name"))
    .dropna()
    .dropDuplicates(["name"])
)

# (Opcional) Nodos de álbum
albums_df = (song_artist
    .select(trim(col("album")).alias("name"))
    .dropna()
    .dropDuplicates(["name"])
)

print("Songs:", songs_df.count())
print("Artists:", artists_df.count())
print("Genres:", genres_df.count())
print("Albums:", albums_df.count())


In [ ]:
# Song -[:PERFORMED_BY]-> Artist
edges_song_artist = (song_artist
    .select(
        col("song_id").alias("src_song_id"),
        col("artist_name").alias("dst_artist_name")
    )
    .dropDuplicates()
)

# Song -[:HAS_GENRE]-> Genre
edges_song_genre = (song_artist
    .select(
        col("song_id").alias("src_song_id"),
        trim(col("genre")).alias("dst_genre_name")
    )
    .dropna()
    .dropDuplicates()
)

# Song -[:PART_OF_ALBUM]-> Album
edges_song_album = (song_artist
    .select(
        col("song_id").alias("src_song_id"),
        trim(col("album")).alias("dst_album_name")
    )
    .dropna()
    .dropDuplicates()
)

print("E(Song->Artist):", edges_song_artist.count())
print("E(Song->Genre):", edges_song_genre.count())
print("E(Song->Album):", edges_song_album.count())


# Writing Data in Neo4j

In [ ]:
# Add the code to write a graph from PySpark's DataFrames to Neo4jneo4j_common = {
    "url": NEO4J_URL,
    "database": NEO4J_DATABASE,
    "authentication.type": "basic",
    "authentication.basic.username": NEO4J_USER,
    "authentication.basic.password": NEO4J_PASSWORD,
}

# ===== NODOS =====
(songs_df.write
 .format("org.neo4j.spark.DataSource")
 .mode("Overwrite")
 .options(**neo4j_common)
 .option("labels", ":Song")
 .option("node.keys", "id")
 .save())

(artists_df.write
 .format("org.neo4j.spark.DataSource")
 .mode("Overwrite")
 .options(**neo4j_common)
 .option("labels", ":Artist")
 .option("node.keys", "name")
 .save())

(genres_df.write
 .format("org.neo4j.spark.DataSource")
 .mode("Overwrite")
 .options(**neo4j_common)
 .option("labels", ":Genre")
 .option("node.keys", "name")
 .save())

(albums_df.write
 .format("org.neo4j.spark.DataSource")
 .mode("Overwrite")
 .options(**neo4j_common)
 .option("labels", ":Album")
 .option("node.keys", "name")
 .save())

print("Nodos escritos en Neo4j.")


In [ ]:
# ===== RELACIONES =====

# Song -[:PERFORMED_BY]-> Artist
(edges_song_artist.write
 .format("org.neo4j.spark.DataSource")
 .mode("Overwrite")
 .options(**neo4j_common)
 .option("relationship", "PERFORMED_BY")
 .option("relationship.save.strategy", "keys")
 .option("relationship.source.labels", ":Song")
 .option("relationship.source.node.keys", "id:src_song_id")
 .option("relationship.target.labels", ":Artist")
 .option("relationship.target.node.keys", "name:dst_artist_name")
 .save())

# Song -[:HAS_GENRE]-> Genre
(edges_song_genre.write
 .format("org.neo4j.spark.DataSource")
 .mode("Overwrite")
 .options(**neo4j_common)
 .option("relationship", "HAS_GENRE")
 .option("relationship.save.strategy", "keys")
 .option("relationship.source.labels", ":Song")
 .option("relationship.source.node.keys", "id:src_song_id")
 .option("relationship.target.labels", ":Genre")
 .option("relationship.target.node.keys", "name:dst_genre_name")
 .save())

# Song -[:PART_OF_ALBUM]-> Album
(edges_song_album.write
 .format("org.neo4j.spark.DataSource")
 .mode("Overwrite")
 .options(**neo4j_common)
 .option("relationship", "PART_OF_ALBUM")
 .option("relationship.save.strategy", "keys")
 .option("relationship.source.labels", ":Song")
 .option("relationship.source.node.keys", "id:src_song_id")
 .option("relationship.target.labels", ":Album")
 .option("relationship.target.node.keys", "name:dst_album_name")
 .save())

print("Relaciones escritas en Neo4j.")


# Read and Query Graphs with PySpark

In [ ]:
# Ejemplo: primeras 20 relaciones Song -> Artist
query = """
MATCH (s:Song)-[r:PERFORMED_BY]->(a:Artist)
RETURN s.id AS song_id, a.name AS artist, type(r) AS rel
LIMIT 20
"""

df_cypher = (spark.read
    .format("org.neo4j.spark.DataSource")
    .options(**neo4j_common)
    .option("query", query)
    .load())

df_cypher.show(truncate=False)


In [1]:
# Top 10 artistas por # de canciones
query_top_artists = """
MATCH (s:Song)-[:PERFORMED_BY]->(a:Artist)
RETURN a.name AS artist, count(*) AS songs
ORDER BY songs DESC
LIMIT 10
"""
top_artists = (spark.read
    .format("org.neo4j.spark.DataSource")
    .options(**neo4j_common)
    .option("query", query_top_artists)
    .load())

top_artists.show(truncate=False)


NameError: name 'spark' is not defined

In [ ]:
sc.stop()